# Transfer Learning

This week we are going to take one of the pre-trained models and adjust it so that it learns to detect our objects instead of the stock 90 objects. This will mean the new model has transfered everything it learned about objects and then learned about our specific object.

Some of the images we labeled last week may not have had pasta sauce jars in them. In that case, we need to delete those images from our folder before we move to the train/test phase.

In [ ]:
import os
import re
from shutil import copyfile
import argparse
import math
import random


def validate_xmls(source):
    source = source.replace('\\', '/')

    images = [f for f in os.listdir(source)
              if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(.jpg|.jpeg|.png)$', f)]

    num_images = len(images)

    for filename in images:
        xml_filename = os.path.splitext(filename)[0]+'.xml'
        if not os.path.exists(os.path.join(source, xml_filename)):
            print("Removing " + filename + "...")
            os.remove(os.path.join(source, filename))
        
validate_xmls('workspace/training_demo/images')

 We'll use the next script to move some of these into `train` and some into `test` for us.

In [ ]:
import os
import re
from shutil import copyfile
import argparse
import math
import random


def iterate_dir(source, dest, ratio, copy_xml):
    source = source.replace('\\', '/')
    dest = dest.replace('\\', '/')
    train_dir = os.path.join(dest, 'train')
    test_dir = os.path.join(dest, 'test')

    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    else:
        for f in os.listdir(train_dir):
            os.remove(os.path.join(train_dir, f))
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
    else:
        for f in os.listdir(test_dir):
            os.remove(os.path.join(test_dir, f))

    images = [f for f in os.listdir(source)
              if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(.jpg|.jpeg|.png)$', f)]

    num_images = len(images)
    num_test_images = math.ceil(ratio*num_images)

    for i in range(num_test_images):
        idx = random.randint(0, len(images)-1)
        filename = images[idx]
        copyfile(os.path.join(source, filename),
                 os.path.join(test_dir, filename))
        if copy_xml:
            xml_filename = os.path.splitext(filename)[0]+'.xml'
            copyfile(os.path.join(source, xml_filename),
                     os.path.join(test_dir,xml_filename))
        images.remove(images[idx])

    for filename in images:
        copyfile(os.path.join(source, filename),
                 os.path.join(train_dir, filename))
        if copy_xml:
            xml_filename = os.path.splitext(filename)[0]+'.xml'
            copyfile(os.path.join(source, xml_filename),
                     os.path.join(train_dir, xml_filename))
            
iterate_dir('workspace/training_demo/images', 'workspace/training_demo/images', 0.1, True)

* Create a new text file in the `training_demo/annotations` folder named `label_map.pbtxt`. Use the Jupyter explorer to open the text file and edit it. It should look like this:

```
item {
    id: 1
    name: 'pasta_sauce_jar'
}
```

where the name is the label you used to identify the objects in `labelimg`.

When it comes time to do multiple labels, your file will look something like this:

```
item {
  id: 1
  name: 'nine'
}

item {
  id: 2
  name: 'ten'
}

item {
  id: 3
  name: 'jack'
}
```

Now we need to convert the XML annotations into tensorflow `*.record` files. This script will do that for us.

In [ ]:
import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow.compat.v1 as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple


def xml_to_csv(path):
    """Iterates through all .xml files (generated by labelImg) in a given directory and combines
    them in a single Pandas dataframe.

    Parameters:
    ----------
    path : str
        The path containing the .xml files
    Returns
    -------
    Pandas DataFrame
        The produced dataframe
    """

    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def class_text_to_int(row_label):
    return label_map_dict[row_label]


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def generate_labels(labels_path, xml_dir, output_path):
    global label_map_dict
    
    label_map = label_map_util.load_labelmap(labels_path)
    label_map_dict = label_map_util.get_label_map_dict(label_map)

    
    writer = tf.python_io.TFRecordWriter(output_path)
    path = os.path.join(xml_dir)
    examples = xml_to_csv(xml_dir)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('Successfully created the TFRecord file: {}'.format(output_path))

        
generate_labels('workspace/training_demo/annotations/label_map.pbtxt',
               'workspace/training_demo/images/train',
               'workspace/training_demo/annotations/train.record')
generate_labels('workspace/training_demo/annotations/label_map.pbtxt',
               'workspace/training_demo/images/test',
               'workspace/training_demo/annotations/test.record')

We now need to go get the pre-trained model we will be using as our base model.

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

We'll use the `SSD ResNet50 V1 FPN 640x640` for our first test. We'll download the file - it is a `.tar.gz` file that will require using a utility program to unzip it. We'll get it unzipped and save it in the `pre-trained-models` folder we created last week. We'll use 7zip to extract this file (https://www.7-zip.org/download.html). The folder structure will look like this when we are done:

```
training_demo/
├─ ...
├─ pre-trained-models/
│  └─ ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/
│     ├─ checkpoint/
│     ├─ saved_model/
│     └─ pipeline.config
└─ ...
```

Now that we have downloaded and extracted our pre-trained model, let’s create a directory for our training job. Under the `training_demo/models` create a new directory named `pastasauce_ssd_resnet50_v1_fpn` and copy the `training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config` file inside the newly created directory. Our training_demo/models directory should now look like this:

```
training_demo/
├─ ...
├─ models/
│  └─ pastasauce_ssd_resnet50_v1_fpn/
│     └─ pipeline.config
└─ ...
```

We update the `pipeline.config` file:

We need to update line 3:

```
    num_classes: 1 # Set this to the number of different label classes
```

Line 131:
```
train_config {
  batch_size: 8 # Increase/Decrease this value depending on the available memory (Higher values require more memory and vice-versa)
```

Line 161:
```
  fine_tune_checkpoint: "pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0" # Path to checkpoint of pre-trained model
```

Lines 167-168:
```
  fine_tune_checkpoint_type: "detection" # Set this to "detection" since we want to be training the full detection model
  use_bfloat16: false # Set this to false if you are not training on a TPU
```

Lines 171-188
```
train_input_reader {
  label_map_path: "annotations/label_map.pbtxt" # Path to label map file
  tf_record_input_reader {
    input_path: "annotations/train.record" # Path to training TFRecord file
  }
}
eval_config {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
}
eval_input_reader {
  label_map_path: "annotations/label_map.pbtxt" # Path to label map file
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "annotations/test.record" # Path to testing TFRecord
  }
}
```

Copy the `internship/models/research/object_detection/model_main_tf2.py` file and paste it straight into your `training_demo` folder.

run `python model_main_tf2.py --model_dir=models/pastasauce_ssd_resnet50_v1_fpn --pipeline_config_path=models/pastasauce_ssd_resnet50_v1_fpn/pipeline.config`

This last one will run for many hours - we will use the tensorboard monitoring to determine when to quit the training. To quit the training, we'll just close the anaconda prompt window that is running the training. This will freeze the training on its last checkpoint.

## Monitoring training

Open a new terminal, activate the virtual environment, navigate to the `training_demo` folder, and run the following command:

`tensorboard --logdir=models/pastasauce_ssd_resnet50_v1_fpn`

The above command will start a new TensorBoard server, which (by default) listens to port 6006 of your machine. Assuming that everything went well, you should see a print-out similar to the one below (plus/minus some warnings):

```...
TensorBoard 2.2.2 at http://localhost:6006/ (Press CTRL+C to quit)
```

Once this is done, go to your browser and type http://localhost:6006/ in your address bar, following which you should be presented with a dashboard.

We'll watch this training for awhile. We're looking for the loss metrics to stabilize- eventually the change will slow down and reach a final point. At this point the training is done and we'll close the terminal to stop the process.

## Exporting the Trained Model

Copy the `internship/models/research/object_detection/exporter_main_v2.py` script and paste it straight into your `training_demo` folder.

From the terminal, go to the `training_demo` folder, run: `python .\exporter_main_v2.py --input_type image_tensor --pipeline_config_path .\models\pastasauce_ssd_resnet50_v1_fpn\pipeline.config --trained_checkpoint_dir .\models\pastasauce_ssd_resnet50_v1_fpn\ --output_directory .\exported-models\pastasaucemodel`

This command may take a few minutes to run as well.

# Testing the model

Our last step is to test this model out on images to see how well it works. We'll need to do a couple of things to import the model and run inference.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


# PATH_TO_SAVED_MODEL = "workspace\\training_demo\\pre-trained-models\\ssd_resnet50_v1_fpn_640x640_coco17_tpu-8\\saved_model"
PATH_TO_SAVED_MODEL = "workspace\\training_demo\\exported-models\\pastasaucemodel\\saved_model"
PATH_TO_LABELS = "workspace\\training_demo\\annotations\\label_map.pbtxt"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))


category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [ ]:
IMAGE_PATHS = ["workspace\\training_demo\\images\\009.jpg",
               "workspace\\training_demo\\images\\032.jpg",
               "workspace\\training_demo\\images\\074.jpg",
              "workspace\\training_demo\\images\\validations\\ftm-pasta-sauce.jpg"]

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings
%matplotlib inline

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


for image_path in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)

    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
plt.show()


# Week 4 Project

Your turn! Set up and train your first model with the data you collected and tagged last week. We want everyone to have a functioning model to demonstrate to the group. Good luck! Because of the holiday, we'll next meet in two weeks. Be prepared to show your functioning model!